In [5]:
#
# classifer notebook
#
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Lambda, Compose
from LandmarkDataset import LandmarkDataset

#
# MODEL
#

class NN(torch.nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, num_classes)
        #self.fc2 = torch.nn.Linear(hidden_size, 48)
        #self.fc3 = torch.nn.Linear(48, num_classes)
        self.dropout = torch.nn.Dropout(p=0.05)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.dropout(out)
        
        out = self.fc2(out)
        #out = self.relu(out)
        #out = self.dropout(out)
        
        #out = self.fc3(out)
        
        return out

    
#
# PARAMS
#

batch_size = 4
hidden_size = 96
learning_rate = .01
num_epochs = 250


#
# INITIAL DATA
#

# for directory load each file
# generate mapping of file -> class -> idx

transformations = Compose([
    Lambda(lambda x: torch.tensor(x.values).float())
])

dataset = LandmarkDataset("/home/jovyan/train/data",
                          "/home/jovyan/model",
                          transform=transformations)

num_classes = dataset.num_class
input_size = dataset.input_size() #2 * (21 * 3) + 12 + 1 + 10 #149

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)



#
# INIT MODELS, LOSS FN, GRAD
#

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NN(input_size, hidden_size, num_classes).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

#
# TRAIN
#

running_loss_epoch = 0
for epoch in range(num_epochs):
    for batch_idx, (labels, landmarks) in enumerate(dataloader):  

        # zero out accumulated gradients
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(landmarks)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        # NB: len(dataloader) is num of batches
        running_loss_epoch += loss.item()
        if ((epoch+1) % 10 == 0) and ((batch_idx+1) % len(dataloader) == 0):
            print ('Epoch [{}/{}], Loss: {:.4f}, Running Loss (per N epoch): {:.4f}' 
                   .format(epoch+1, num_epochs, loss.item(), running_loss_epoch))
            running_loss_epoch = 0


Epoch [10/250], Loss: 0.6908, Running Loss (per N epoch): 1054.1643
Epoch [20/250], Loss: 1.0850, Running Loss (per N epoch): 721.8633
Epoch [30/250], Loss: 1.6471, Running Loss (per N epoch): 639.9415
Epoch [40/250], Loss: 1.2729, Running Loss (per N epoch): 610.4878
Epoch [50/250], Loss: 1.2773, Running Loss (per N epoch): 567.8488
Epoch [60/250], Loss: 0.8200, Running Loss (per N epoch): 562.4134
Epoch [70/250], Loss: 0.5815, Running Loss (per N epoch): 552.9380
Epoch [80/250], Loss: 0.5054, Running Loss (per N epoch): 509.3956
Epoch [90/250], Loss: 0.3743, Running Loss (per N epoch): 503.6390
Epoch [100/250], Loss: 0.5848, Running Loss (per N epoch): 536.9972
Epoch [110/250], Loss: 0.2387, Running Loss (per N epoch): 578.4746
Epoch [120/250], Loss: 0.6983, Running Loss (per N epoch): 533.3518
Epoch [130/250], Loss: 0.1875, Running Loss (per N epoch): 541.9587
Epoch [140/250], Loss: 0.6952, Running Loss (per N epoch): 517.7738
Epoch [150/250], Loss: 0.3625, Running Loss (per N epoch

In [6]:
#
# EXPORT
#
# NB: LandmarkDataset.py updates meta.json with class index
#
dummy_input = torch.zeros(input_size)
#model.load_state_dict(torch.load('./model_overfit.pt'))
torch.onnx.export(model, dummy_input, 'onnx_model.onnx', export_params=True,
                  input_names = ['landmarks'], output_names = ['class'], verbose=True)

#import onnx
#onnx_model = onnx.load("./onnx_model.onnx")
#onnx.checker.check_model(onnx_model)


Exported graph: graph(%landmarks : Float(138, strides=[1], requires_grad=0, device=cpu),
      %fc1.bias : Float(96, strides=[1], requires_grad=1, device=cpu),
      %fc2.bias : Float(16, strides=[1], requires_grad=1, device=cpu),
      %onnx::MatMul_12 : Float(138, 96, strides=[1, 138], requires_grad=0, device=cpu),
      %onnx::MatMul_13 : Float(96, 16, strides=[1, 96], requires_grad=0, device=cpu)):
  %onnx::Add_6 : Float(96, strides=[1], device=cpu) = onnx::MatMul[onnx_name="MatMul_0"](%landmarks, %onnx::MatMul_12) # /opt/conda/lib/python3.10/site-packages/torch/nn/modules/linear.py:114:0
  %input : Float(96, strides=[1], requires_grad=1, device=cpu) = onnx::Add[onnx_name="Add_1"](%fc1.bias, %onnx::Add_6) # /opt/conda/lib/python3.10/site-packages/torch/nn/modules/linear.py:114:0
  %input.3 : Float(96, strides=[1], requires_grad=1, device=cpu) = onnx::Relu[onnx_name="Relu_2"](%input) # /opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1453:0
  %onnx::Add_10 : Float(16, 

In [7]:
#
# EVAL
#
torch.set_printoptions(precision=4, sci_mode=False)
softmax = torch.nn.Softmax(dim=1)
accuracy = 0
count = 0
with torch.no_grad():
    
    for batch_idx, (labels, landmarks) in enumerate(dataloader):  
        out = model(landmarks)
        prob = softmax(out.data)     #setup for threshold or 'garbage' class
        _, klass = torch.max(out.data, 1)
        
        print(klass, labels, klass==labels)
        print(prob)
        print("-----")
        
        accuracy += (klass == labels).sum().item()
        count += len(labels)
        

print('--------')
print("Accuracy {}/{} : {:.4f}".format(accuracy, count, accuracy/count))


tensor([ 2,  7,  9, 14]) tensor([ 2,  7, 10, 14]) tensor([ True,  True, False,  True])
tensor([[    0.0000,     0.0000,     0.9143,     0.0187,     0.0000,     0.0003,
             0.0003,     0.0636,     0.0019,     0.0000,     0.0000,     0.0010,
             0.0000,     0.0000,     0.0000,     0.0000],
        [    0.0000,     0.0000,     0.0936,     0.0467,     0.0000,     0.0002,
             0.0022,     0.8241,     0.0262,     0.0000,     0.0000,     0.0070,
             0.0000,     0.0000,     0.0000,     0.0000],
        [    0.0000,     0.0000,     0.0002,     0.0014,     0.0458,     0.0050,
             0.0219,     0.0035,     0.0076,     0.4730,     0.2990,     0.1425,
             0.0000,     0.0000,     0.0000,     0.0000],
        [    0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
             0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
             0.0000,     0.0000,     1.0000,     0.0000]])
-----
tensor([ 0,  4, 13,  4]) t

In [8]:
print("Done")

Done
